In [1]:
import pandas as pd
from pandas import json_normalize
from tqdm import tqdm
import time
import os
import requests
from bs4 import BeautifulSoup
import json

In [141]:
file_list = os.listdir()

xlsx_list = [xlsx for xlsx in file_list if xlsx.endswith(".xlsx")]

df = pd.read_excel('./{0}'.format(xlsx_list[0]))
df = df.rename(columns={'월별_수출(천$)':'월별'})

df

,월별,기간,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
0,201601,1-10,8453665,1108532,699490,267042,421461,566581,857006,404300,139216,147506,128526
1,201601,1-20,22081425,2798444,1713597,1120283,993571,1197717,2302532,1096189,338288,376100,430450
2,201601,1-31,36260420,4590675,2963953,2773742,1684516,1964416,2617192,1871217,552120,597448,696229
3,201602,1-10,8723002,1148994,738513,330477,483390,427479,360863,530185,156696,154552,210700
4,201602,1-20,21967139,2662160,1777030,1234504,1123759,1301971,1512134,1188557,358157,368288,451911
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,202204,1-31,57858183,11001497,5078309,4199277,5184185,1832287,951092,1889161,1807617,1077705,1063313
228,202205,1-10,16042304,2798732,1233662,596229,2366601,460960,396621,506005,337158,295724,311464
229,202205,1-20,38547511,6608142,3216826,2224359,4433804,1039848,1188052,1258517,976616,708345,700464
230,202205,1-31,61509507,11716953,5155695,3909811,6435808,1491320,1932894,1894433,1799911,1082749,1067522


In [181]:
# 가장 마지막 행 데이터의 기간과 일치
temp_df = df.loc[df['기간'] == df.iloc[-1,1]]

temp_df

,월별,기간,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
0,201601,1-10,8453665,1108532,699490,267042,421461,566581,857006,404300,139216,147506,128526
3,201602,1-10,8723002,1148994,738513,330477,483390,427479,360863,530185,156696,154552,210700
6,201603,1-10,11969107,1498350,1218726,520020,511333,783072,711249,685814,182477,199171,255414
9,201604,1-10,10525943,1182806,973800,470934,711169,577894,716900,587023,148237,166011,194156
12,201605,1-10,9306549,1278394,744731,246433,402019,641028,540579,517236,134199,172975,206426
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,202202,1-10,15750415,3298814,1261971,653020,1358897,472115,457938,512383,378033,347262,332503
222,202203,1-10,18829899,3846223,1549104,725851,1837928,586415,252396,711328,467347,409517,375877
225,202204,1-10,15356135,2940578,1223891,921416,1489141,663130,260796,472671,305562,308992,299014
228,202205,1-10,16042304,2798732,1233662,596229,2366601,460960,396621,506005,337158,295724,311464


In [185]:
temp_df_2 = df.groupby(['월별', '기간'])
temp_df_2 = temp_df_2.first()

temp_df_2

전체       반도체     철강제품    승용자동차     석유제품   무선통신기기       선박  \
월별     기간                                                                      
201601 1-10   8453665   1108532   699490   267042   421461   566581   857006   
       1-20  22081425   2798444  1713597  1120283   993571  1197717  2302532   
       1-31  36260420   4590675  2963953  2773742  1684516  1964416  2617192   
201602 1-10   8723002   1148994   738513   330477   483390   427479   360863   
       1-20  21967139   2662160  1777030  1234504  1123759  1301971  1512134   
...               ...       ...      ...      ...      ...      ...      ...   
202204 1-31  57858183  11001497  5078309  4199277  5184185  1832287   951092   
202205 1-10  16042304   2798732  1233662   596229  2366601   460960   396621   
       1-20  38547511   6608142  3216826  2224359  4433804  1039848  1188052   
       1-31  61509507  11716953  5155695  3909811  6435808  1491320  1932894   
202206 1-10  15069121   3165477  1217600   623906  1877693   495337   388974   

               자동차부품  컴퓨터주변기기     정밀기기     가전제품  
월별     기간                                        
201601 1-10   404300   139216   147506   128526  
       1-20  1096189   338288   376100   430450  
       1-31  1871217   552120   597448   696229  
201602 1-10   530185   156696   154552   210700  
       1-20  1188557   358157   368288   451911  
...              ...      ...      ...      ...  
202204 1-31  1889161  1807617  1077705  1063313  
202205 1-10   506005   337158   295724   311464  
       1-20  1258517   976616   708345   700464  
       1-31  1894433  1799911  1082749  1067522  
202206 1-10   497651   286011   270921   215215  

[232 rows x 11 columns]

In [188]:
temp_df_2.loc[202205]

,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
기간,,,,,,,,,,,
1-10,16042304,2798732,1233662,596229,2366601,460960,396621,506005,337158,295724,311464
1-20,38547511,6608142,3216826,2224359,4433804,1039848,1188052,1258517,976616,708345,700464
1-31,61509507,11716953,5155695,3909811,6435808,1491320,1932894,1894433,1799911,1082749,1067522


In [192]:
temp_df_2.xs('1-31', level='기간')

,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
월별,,,,,,,,,,,
201601,36260420,4590675,2963953,2773742,1684516,1964416,2617192,1871217,552120,597448,696229
201602,35924541,4191505,2973930,2724255,1703253,2071924,2621257,1834848,528011,558704,715932
201603,43002468,5328931,3914314,3683674,1915398,2614236,2516427,2179476,609178,705732,862126
201604,41081553,4590248,3265409,3231914,2435747,2306875,3421545,2033711,557562,621048,757022
201605,39733794,4923303,3149374,3166111,2193445,2390987,1865317,2018372,596228,652761,815976
...,...,...,...,...,...,...,...,...,...,...,...
202201,55457577,11015767,5060251,3924570,3724347,1663892,724661,1974518,1529994,1154352,1177138
202202,54008302,10545061,4663201,3633034,4198826,1664397,1703335,1727889,1629532,1145982,1040642
202203,63780433,13297661,5075736,3783383,5534770,2242741,1454676,2111906,1876376,1312426,1176590


In [174]:
temp_df_2 = df.loc[df['기간'] == '1-31']

temp_df_2

,월별,기간,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
2,201601,1-31,36260420,4590675,2963953,2773742,1684516,1964416,2617192,1871217,552120,597448,696229
5,201602,1-31,35924541,4191505,2973930,2724255,1703253,2071924,2621257,1834848,528011,558704,715932
8,201603,1-31,43002468,5328931,3914314,3683674,1915398,2614236,2516427,2179476,609178,705732,862126
11,201604,1-31,41081553,4590248,3265409,3231914,2435747,2306875,3421545,2033711,557562,621048,757022
14,201605,1-31,39733794,4923303,3149374,3166111,2193445,2390987,1865317,2018372,596228,652761,815976
...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,202201,1-31,55457577,11015767,5060251,3924570,3724347,1663892,724661,1974518,1529994,1154352,1177138
221,202202,1-31,54008302,10545061,4663201,3633034,4198826,1664397,1703335,1727889,1629532,1145982,1040642
224,202203,1-31,63780433,13297661,5075736,3783383,5534770,2242741,1454676,2111906,1876376,1312426,1176590
227,202204,1-31,57858183,11001497,5078309,4199277,5184185,1832287,951092,1889161,1807617,1077705,1063313


In [201]:
temp_df_3 = df[(df['기간'] == '1-31') & df['월별'].between(202200, 202300)]

temp_df_3

,월별,기간,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
218,202201,1-31,55457577,11015767,5060251,3924570,3724347,1663892,724661,1974518,1529994,1154352,1177138
221,202202,1-31,54008302,10545061,4663201,3633034,4198826,1664397,1703335,1727889,1629532,1145982,1040642
224,202203,1-31,63780433,13297661,5075736,3783383,5534770,2242741,1454676,2111906,1876376,1312426,1176590
227,202204,1-31,57858183,11001497,5078309,4199277,5184185,1832287,951092,1889161,1807617,1077705,1063313
230,202205,1-31,61509507,11716953,5155695,3909811,6435808,1491320,1932894,1894433,1799911,1082749,1067522


In [134]:
def calc_division(a, b):
    c = (b/a-1)*100
    return c

import sys

# sys모듈을 통한 동적변수 생성 ('object_연도'로 파일리스트 포함한 변수 생성됨)
mod = sys.modules[__name__]



for row in temp_df.itertuples():
    
    if 전체 != 0:
        print(calc_division(전체, row.전체))
        
    i = row.index
    month = row.월별
    전체 = row.전체
    반도체 = row.반도체
    철강제품 = row.철강제품
    승용자동차 = row.승용자동차
    석유제품 = row.석유제품
    무선통신기기 = row.무선통신기기
    선박 = row.선박
    자동차부품 = row.자동차부품
    컴퓨터주변기기 = row.컴퓨터주변기기
    정밀기기 = row.정밀기기
    가전제품 = row.가전제품
    
    print(month)

-43.90074245206472
201601
3.186038244950562
201602
37.21316354163395
201603
-12.057407457381741
201604
-11.58465327049557
201605
49.18529951327824
201606
-25.807240174177437
201607
-5.854250953191098
201608
39.130859796926806
201609
-29.95429841274735
201610
46.710527568777785
201611
-7.396040198059328
201612
-9.397613500321022
201701
29.501546088284456
201702
-6.0307928688814965
201703
-21.201776866853795
201704
-13.477711053383768
201705
25.689566929133868
201706
17.37005398400222
201707
-12.61461013434053
201708
-1.104548197312405
201709
-45.414409458628455
201710
139.3272700934706
201711
-21.87130022548873
201712
8.945169946460929
201801
7.566940546469425
201802
-7.994346344106884
201803
2.016410869145413
201804
0.8373477797909423
201805
-11.452756413380927
201806
12.826207081727725
201807
6.359438050973831
201808
-5.4373396765731385
201809
2.5646931555005414
201810
4.560694654650899
201811
-20.447257314818646
201812
5.8197254436002765
201901
-50.01989058004761
201902
73.2211607579

In [123]:
import sys

# sys모듈을 통한 동적변수 생성 ('object_연도'로 파일리스트 포함한 변수 생성됨)
mod = sys.modules[__name__]

for a in temp_df:
    # print(a)
    setattr(mod, 'object_{}'.format(a), 0)